<a href="https://colab.research.google.com/github/plklinh/ECON_599_Team_1/blob/master/FB_Prophet_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd drive/My\ Drive/U\ of\ C/ECON_599/ECON_599_Team_1

/content/drive/My Drive/U of C/ECON_599/ECON_599_Team_1


In [ ]:
import numpy as np 
import pandas as pd 
import holidays
from fbprophet import Prophet 
from sklearn.model_selection import TimeSeriesSplit
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [ ]:
data = pd.read_csv("msa_merged_data.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,BEGIN_DATE_GMT,HE,POOL_PRICE,AIL_DEMAND,Avg_temp,Weighted_Avg_Temp,future 1,future 2,future 3,future 4,WTI spot,dayofweek,month,year,holiday,workingday
0,0,0,2010-01-01 07:00:00,1,51.12,8307.0,-24.100,-22.952601,79.36,80.02,80.63,81.11,79.39,4,1,2010,1,0
1,1,1,2010-01-01 08:00:00,2,48.79,8186.0,-23.775,-22.588126,79.36,80.02,80.63,81.11,79.39,4,1,2010,1,0
2,2,2,2010-01-01 09:00:00,3,39.56,8075.0,-23.425,-22.339761,79.36,80.02,80.63,81.11,79.39,4,1,2010,1,0
3,3,3,2010-01-01 10:00:00,4,36.27,8013.0,-23.000,-21.877969,79.36,80.02,80.63,81.11,79.39,4,1,2010,1,0
4,4,4,2010-01-01 11:00:00,5,36.16,7982.0,-22.400,-21.527531,79.36,80.02,80.63,81.11,79.39,4,1,2010,1,0


In [ ]:
# Converting Pool_price to float - no need to rerun
data["POOL_PRICE"] = data["POOL_PRICE"].replace(" -   ", np.NaN).replace("[\s,]", "", regex=True)
data["POOL_PRICE"] = pd.to_numeric( data["POOL_PRICE"] )
data.dtypes

BEGIN_DATE_GMT       datetime64[ns]
HE                            int64
POOL_PRICE                  float64
AIL_DEMAND                  float64
Avg_temp                    float64
Weighted_Avg_Temp           float64
future 1                    float64
future 2                    float64
future 3                    float64
future 4                    float64
WTI spot                    float64
dayofweek                     int64
month                         int64
year                          int64
holiday                       int64
workingday                    int64
dtype: object

In [ ]:
# Checking for null values
print(data.shape)
data.isnull().sum()

(96390, 17)


Unnamed: 0             0
BEGIN_DATE_GMT         0
HE                     0
POOL_PRICE           150
AIL_DEMAND             0
Avg_temp               0
Weighted_Avg_Temp      0
future 1               0
future 2               0
future 3               0
future 4               0
WTI spot               0
dayofweek              0
month                  0
year                   0
holiday                0
workingday             0
dtype: int64

In [ ]:
# Converting to datetime and checking for gaps in the the time series
data["BEGIN_DATE_GMT"] = pd.to_datetime(data["BEGIN_DATE_GMT"])
sum(data["BEGIN_DATE_GMT"] - data["BEGIN_DATE_GMT"].shift(1) > pd.Timedelta(1, unit = "h"))

# Drop unused columns
data.drop(columns = ["Unnamed: 0", "Unnamed: 0.1"], inplace = True)

In [ ]:
# import holidays
# canada_holidays = holidays.CA()
# data["holiday"] = [1 if i.date() in canada_holidays else 0 for i in data["BEGIN_DATE_GMT"]]
# data["workingday"] = data.apply(lambda row: 0 if row["holiday"] == 1 or row["dayofweek"] in [5,6] else 1 , axis = 1)
# data.to_csv("msa_merged_data.csv")

In [ ]:
# Train-test split
y = data["AIL_DEMAND"]
X = data.drop(columns = ["AIL_DEMAND"])

In [ ]:
# sk_learn corss validation technique
tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)
for train_index, test_index in tscv.split(X):
    print("TRAIN: ", train_index.max(), "TEST: (", test_index.min(), test_index.max(), ")")

TRAIN:  16064 TEST: ( 16065 32129 )
TRAIN:  32129 TEST: ( 32130 48194 )
TRAIN:  48194 TEST: ( 48195 64259 )
TRAIN:  64259 TEST: ( 64260 80324 )
TRAIN:  80324 TEST: ( 80325 96389 )


In [ ]:
# Making FBProphet Datasets
def make_prophet_df(X, y, regressors):
    data = pd.DataFrame()
    data["y"] = y
    data['ds'] = X["BEGIN_DATE_GMT"]
    for i in regressors:
        data[i] = X[i]
    return data


# Prophet model
prophet_model = Prophet(growth='linear', interval_width = 0.95, 
                yearly_seasonality='auto',
                weekly_seasonality='auto',
                daily_seasonality='auto',
                seasonality_mode='additive'
              )
# Adding regressors
regressors = ["POOL_PRICE", "Weighted_Avg_Temp", "workingday"]
# Continous vars
prophet_model.add_regressor(regressors[0], 
                    prior_scale=20, mode='additive', standardize=True)
prophet_model.add_regressor(regressors[1], 
                    #prior_scale = 1, 
                    mode='additive',
                    standardize=True)
# Binary vars  
prophet_model.add_regressor(regressors[2], 
                    #prior_scale=10, 
                    mode='additive', 
                    standardize='auto') 

# FB Prophet cross validation techique
split_index = np.int(0.2* len(y))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

prop_train_df = make_prophet_df(X_train, y_train, regressors)
prop_train_df["POOL_PRICE"] = prop_train_df["POOL_PRICE"].interpolate(method = "cubic")

prophet_model.fit(prop_train_df)

prop_cv = cross_validation(prophet_model, horizon = 16064 ) 
prop_p = performance_metrics(prop_cv)

prop_p.head()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Making 10512 forecasts with cutoffs between 2011-01-01 13:59:59.999983936 and 2012-03-14 13:59:59.999983936


In [ ]:
prop_test = make_prophet_df(X_test, y_test)
prop_test.drop("y", inplace = True)

# Creating the dataframe with datetime values to predict on (making predictions on train as well as the test set)
future_dates = prophet_model.make_future_dataframe(periods=len(data_test), freq='H', include_history=True)
# Aadding regressors 
future_dates = pd.merge(future_dates, (prop_train_df.append(prop_test)).drop('y', axis=1), on = 'ds')

# Predicting the future
forecast = prop.predict(future_dates)

prophet_model.plot(forecast, uncertainty = True, 
          xlabel = 'Dates', ylabel = 'Alberta Internal Load MWH')
prophet_model.plot_components(forecast)

NameError: ignored

In [ ]:
(32129-16065 )*3

48192